# Sliding demonstration
This notebook demonstrates how our simulation can model ribbons sliding past each other when segment rest lengths and rest kappas are added as variables.
We slide a wavey ribbon through a small grid of ribbons by sampling each ribbon segment's rest centerline from the corresponding intervals of a parametric reference wavey ribbon.

In the following, we consider the wavey ribbon to be oriented vertically. We parametrize it by *height* rather than by arc length to precisely
control the ribbon's vertical extent between each of the grid's horizontal ribbons.
After sliding the ribbon by distance d, the ribbon extends from (0.5 - d) units above the grid's top horizontal ribbon to (0.5 + d) units below.

Placing the bottom of the grid at $y=0$, the individual ribbon segments then cover the $y$ intervals:
```
[-d, 0.5]
...
[i - 0.5, i + 0.5] # (for i in 1...gridEdges - 2 inclusive)
...
[gridEdges - 1.5, gridEdges - 1 - d]
```
However, because the ribbon extends past a crossing by half an edge, the true endpoints of the ribbon segments are roughly:
```
[-d, 0.5 + 0.5 * (0.5 + d) / (n_s - 0.5)],           # length of each DER edge is roughly (0.5 + d) / (ns - 0.5)
...
[i - 0.5 - 0.5 / (ns - 1), i + 0.5 + 0.5 / (ns - 1)] # length of each DER edge is roughly 1 / (ns - 1)
...
[gridEdges - 1.5 - 0.5 * (0.5 - d) / (n_s - 0.5), gridEdges - 1 - d] # length of each DER edge is roughly (0.5 - d) / (ns - 0.5)
```
Then these $y$ intervals are mapped into curve parameter $t = y + d$.

In [1]:
import sys; sys.path.extend(['..', '../../elastic_rods/python'])
from matplotlib import pyplot as plt

In [2]:
import elastic_rods
import numpy as np
import mesh_operations

In [3]:
nopts = elastic_rods.NewtonOptimizerOptions()
nopts.useNegativeCurvatureDirection = False
nopts.gradTol = 1e-10

In [4]:
curvyCenterline = lambda t: np.array([0.2 * np.sin(np.pi * t), t, 0]) # wavey ribbon evaluated at parametric "height" t

In [5]:
gridSize = 5
gridEdges = gridSize - 1
gridCoords = [0] + list(0.5 + np.arange(gridEdges - 1)) + [gridEdges - 1]
horiz = np.column_stack([np.arange(gridEdges), 1 + np.arange(gridEdges)])
vert = gridSize * horiz
V = np.array([[x, y, 0] for x in gridCoords for y in gridCoords])
E = np.vstack([horiz + i * gridSize for i in range(1, gridSize - 1)] + [vert + i for i in range(1, gridSize - 1)])
V, E = mesh_operations.removeDanglingVertices(V, E)

In [6]:
import MeshFEM, tri_mesh_viewer
import linkage_vis

In [7]:
d = 0

In [8]:
n_s = 30
#itype = elastic_rods.InterleavingType.weaving
itype = elastic_rods.InterleavingType.noOffset
l = elastic_rods.RodLinkage(V, E, rod_interleaving_type=itype, subdivision=n_s)
mat = elastic_rods.RodMaterial('rectangle', 200, 0.3, [0.04, 0.01])
l.setMaterial(mat)

In [9]:
lv = linkage_vis.getColoredRodOrientationViewer(l, topColor=[79/255., 158/255., 246/255.])
lv.averagedMaterialFrames = True
lv.averagedCrossSections = True
lv.show()

Renderer(camera=PerspectiveCamera(children=(PointLight(color='white', intensity=0.6, position=(0.0, 0.0, 5.0),…

In [10]:
lv.update()

In [11]:
lv.showWireframe()

In [12]:
mat = elastic_rods.RodMaterial('rectangle', 200, 0.3, [0.04, 0.01])
mat.twistingStiffness *= 1
l.setMaterial(mat)

In [13]:
# Pick one of the ribbons to make wavey
ribbon = l.traceRods()[1][1]

In [14]:
# Set the rest shape by sampling the appropriate intervals of the reference curve

In [15]:
def heightIntervalForSegment(i):
    if i == 0:             return [-d, 0.5 + 0.5 * (0.5 + d) / (n_s - 0.5)]                            # length of each DER edge is roughly (0.5 + d) / (ns - 0.5)
    if i < gridEdges - 1:  return [i - 0.5 - 0.5 / (n_s - 1), i + 0.5 + 0.5 / (n_s - 1)]               # length of each DER edge is roughly         1 / (ns - 1)
    if i == gridEdges - 1: return [gridEdges - 1.5 - 0.5 * (0.5 - d) / (n_s - 0.5), gridEdges - 1 - d] # length of each DER edge is roughly (0.5 - d) / (ns - 0.5)
    raise Exception('index out of range')

In [16]:
for i, sidx in enumerate(ribbon):
    r = l.segment(sidx).rod
    curvyRod = elastic_rods.ElasticRod(np.array([curvyCenterline(t - d) for t in np.linspace(*heightIntervalForSegment(i), num=(n_s + 1))]))
    #print(np.array(curvyRod.restKappas()))
    r.setRestKappas(curvyRod.restKappas())
    #r.setRestLengths(curvyRod.restLengths())

In [17]:
E0 = 1
l0 = 1

In [18]:
l.set_design_parameter_config(True, False)

In [19]:
fv = [o for ji in range(l.numJoints()) for o in l.dofOffsetForJoint(ji)
          + np.arange(0 if (ji < 3 or ji > 5) else 3, 6)] # Fix all joint orientations and joint positions 0, 1, 2, 6, 7, 8
#elastic_rods.restlen_solve(l, fixedVars=fv, options=nopts)#fv, options=nopts)
elastic_rods.designParameter_solve(l, fixedVars=[], options=nopts, regularization_weight=0, smoothing_weight=0, E0=E0, l0=l0)
lv.update()

0	0.00101525	0.0762543	0.0762543	1	1
1	5.651e-05	0.00199804	0.00199804	1	1
2	3.04506e-05	0.000492472	0.000492472	1	1
3	2.67072e-05	0.000121182	0.000121182	1	1
4	2.64537e-05	2.55433e-05	2.55433e-05	1	1
5	2.64456e-05	1.8515e-05	1.8515e-05	1	1
6	2.64448e-05	1.84592e-05	1.84592e-05	1	1
7	2.6444e-05	1.84696e-05	1.84696e-05	1	1
8	2.64437e-05	1.84789e-05	1.84789e-05	1	1
9	2.64432e-05	1.84875e-05	1.84875e-05	1	1
10	2.64431e-05	1.84904e-05	1.84904e-05	1	1
11	2.64428e-05	1.84875e-05	1.84875e-05	1	1
12	2.64427e-05	1.84847e-05	1.84847e-05	1	1
13	2.64424e-05	1.84743e-05	1.84743e-05	1	1
14	2.64423e-05	1.84684e-05	1.84684e-05	1	1
15	2.64422e-05	1.84543e-05	1.84543e-05	1	1
16	2.64421e-05	1.8447e-05	1.8447e-05	1	1
17	2.64419e-05	1.84314e-05	1.84314e-05	1	1
18	2.64418e-05	1.84236e-05	1.84236e-05	1	1
19	2.64417e-05	1.84077e-05	1.84077e-05	1	1
20	2.64416e-05	1.83998e-05	1.83998e-05	1	1
21	2.64415e-05	1.8384e-05	1.8384e-05	1	1
22	2.64415e-05	1.83762e-05	1.83762e-05	1	1
23	2.64414e-05	1.83606e-05	1.83606e-0

## Debugging: reconstruct a single rod from prescribed curvature/rest length
We can correctly reconstruct the state of a rod from its rest curvature/length,
including the individual ribbon segments extracted from the linkage...

The call `getRod(-1)` gets a custom example with known ground truth, while `getRod(0...4)` takes a ribbon segment from the linkage.

In [20]:
import copy
def getRod(ri):
    global groundTruth
    groundTruth = None
    if ri >= 0: return copy.deepcopy(l.segment(ribbon[ri]).rod)
    
    curvyCenterline = lambda t: np.array([0.4 * np.sin(np.pi * t), t, 0]) # wavey ribbon evaluated at parametric "height" t
    arclens = np.linspace(0, 4, 100)
    groundTruth = elastic_rods.ElasticRod(np.array([curvyCenterline(s) for s in arclens]))
    r = elastic_rods.ElasticRod(np.pad(arclens[:, np.newaxis], [(0, 0), (0, 2)]))

    mat = elastic_rods.RodMaterial('rectangle', 200, 0.3, [0.05, 0.01])
    r.setMaterial(mat)
    groundTruth.setMaterial(mat)

    r.setRestKappas(np.array(groundTruth.restKappas()))
    r.setRestLengths(groundTruth.restLengths())
    return r

In [21]:
r = getRod(-1)

In [22]:
rv = tri_mesh_viewer.TriMeshViewer(r)
rv.show()

Renderer(camera=PerspectiveCamera(children=(PointLight(color='white', intensity=0.6, position=(0.0, 0.0, 5.0),…

In [23]:
rv.showWireframe()

In [24]:
# When the correct rest leng set, we can actually reconstruct the ribbon with just a `compute_equilibrium` call.
# # Rerun this until convergence
# elastic_rods.compute_equilibrium(r, options=nopts, fixedVars=[0, 1, 2])
# rv.update()

In [25]:
E0 = 1
l0 = 1

r.set_design_parameter_config(True, False)
#elastic_rods.restlen_solve(r, options=nopts, fixedVars=[0, 1, 2]) # this seems broken; changes rest curvature even if we tell it not to...
firstPoint = [0, 1, 2]
lastPoint = list(r.thetaOffset() + np.array([-3, -2, -1]))
fv = firstPoint + lastPoint
#fv = firstPoint
elastic_rods.designParameter_solve(r, options=nopts, fixedVars=fv, smoothing_weight=0, regularization_weight=0, E0=E0, l0=l0) # This doesn't work well either.
rv.update()

0	0.0199136	0.217453	0.217453	1	1
1	0.0197913	0.216811	0.216811	1	1
2	0.019732	0.216509	0.216509	1	1
3	0.0196184	0.215953	0.215953	1	1
4	0.0195631	0.215691	0.215691	1	1
5	0.0194569	0.215204	0.215204	1	1
6	0.0194051	0.214974	0.214974	1	1
7	0.0193053	0.214543	0.214543	1	1
8	0.0192567	0.214339	0.214339	1	1
9	0.0191626	0.213956	0.213956	1	1
10	0.0191166	0.213773	0.213773	1	1
11	0.0190277	0.21343	0.21343	1	1
12	0.0189842	0.213266	0.213266	1	1
13	0.0188998	0.212957	0.212957	1	1
14	0.0188586	0.212809	0.212809	1	1
15	0.0187784	0.21253	0.21253	1	1
16	0.0187392	0.212396	0.212396	1	1
17	0.0186629	0.212143	0.212143	1	1
18	0.0186256	0.212022	0.212022	1	1
19	0.0185529	0.211792	0.211792	1	1
20	0.0185172	0.211681	0.211681	1	1
21	0.0184478	0.211472	0.211472	1	1
22	0.0184138	0.211371	0.211371	1	1
23	0.0183475	0.21118	0.21118	1	1
24	0.0183149	0.211088	0.211088	1	1
25	0.0182515	0.210913	0.210913	1	1
26	0.0182203	0.210829	0.210829	1	1
27	0.0181595	0.210668	0.210668	1	1
28	0.0181297	0.210591	0.210591	1	1
29

In [26]:
# Compare to ground-truth rod
if groundTruth is not None:
    import registration
    V, F, N = groundTruth.visualizationGeometry()
    R, t = registration.register_points(r.visualizationGeometry()[0], V)
    V = V @ R.T + t
    rv.update(mesh=tri_mesh_viewer.RawMesh(V, F), preserveExisting=True)